In [1]:
try:
    import piplite
    await piplite.install(["ipywidgets==8.1.0", "ipyleaflet==0.17.3", "emfs:here_search_demo-0.9.1-py3-none-any.whl"], keep_going=True)
    api_key = "<YOUR HERE API KEY>"
except ImportError:
    api_key = None

# Input widgets

To interact with the [HERE search & geocoding API](https://developer.here.com/documentation/geocoding-search-api/api-reference-swagger.html) services `here_search` provides a few widgets based on [`ipywidgets`](https://ipywidgets.readthedocs.io/en/stable/index.html):
- `SubmittableTextBox` and `TermsButtons` to formulate textual search intents for Autosuggest and Discover
- `PlaceTaxonomyButtons` to formulate custom place classification shortcuts for Browse
- `OneBoxSimple` to associate search intents to search requests

Those widgets are using a queue to exchange objects of `SearchIntent` class derivatives.

### SubmittableTextBox

`SubmittableTextBox` is a special `Text` widget supporting key strokes events and text submissions.<br/>
It uses a queue to post `SearchIntent` objects, either `TransientTextIntent` for each keystroke or `FornulatedTextIntent` when the transient text is submitted. Search intent is materialized in these objects through their `materialization` attribute, set to a string for these two later classes.

In the snippet below, key strokes and text submissions are displayed in an `Output` widget.

In [ ]:
from here_search.demo.widgets.input import SubmittableTextBox
from here_search.demo.widgets.util import Output
from here_search.demo.entity.intent import FormulatedTextIntent
from asyncio import ensure_future, Queue

box = SubmittableTextBox(Queue())
out_keys, out_text = Output(height=30), Output(height=30)

async def print_text():
    while True:
        intent = await box.queue.get()
        out = out_text if isinstance(intent, FormulatedTextIntent) else out_keys
        with out:
            out.replace(intent.materialization)

t1 = ensure_future(print_text())
display(box, out_keys, out_text)

### TermsButtons

`TermsButtons` objects are taking a `SubmittableTextBox` instance to feed it with predefined values when the buttons are selected.<br/>
Per default, the complete form value is overwritten by the button text. 

The constructor `index` parameter allows to specify wich token is overwritten:

In [ ]:
from here_search.demo.widgets.input import TermsButtons

box2 = SubmittableTextBox(Queue())
out_keys2 = Output(height=30)

async def print_text2():
    while True:
        intent = await box2.queue.get()
        with out_keys2:
            out_keys2.replace(intent.materialization)
            
terms = TermsButtons(box2, 
                     values=["text 1", "text 2"], 
                     index=-1)

t2 = ensure_future(print_text2())
display(box2, terms, out_keys2)

### PlaceTaxonomyButtons

`PlaceTaxonomyButtons` is a widget associating a list of buttons to a place classification.<br/>
It uses a queue to post a `PlaceTaxonomyIntent` object materialized with the place taxonomy item when a button is selected.

The example belows uses the `PlaceTaxonomyExample` object. Each buttons selection displays the related taxonomy item in an `Output` widget.

(fontawesome 5.12.0 [download link](https://use.fontawesome.com/releases/v5.12.0/fontawesome-free-5.12.0-web.zip))

In [ ]:
from here_search.demo.entity.place import PlaceTaxonomyExample
from here_search.demo.widgets.input import PlaceTaxonomyButtons

buttons = PlaceTaxonomyButtons(Queue(), PlaceTaxonomyExample.taxonomy, PlaceTaxonomyExample.icons)
out_onto = Output(height=30)

async def print_taxonomy():
    while True:
        with out_onto:
            intent = await buttons.queue.get()
            out_onto.replace(str(intent.materialization))
            
t4 = ensure_future(print_taxonomy())
display(buttons, out_onto)

## ResponseMap

Search end-users fire requests (text queries, or ontology shortcuts) from a particular location context, materialized in the API through the search center lat/lon. It is convenient to capture a search center thrioygh the use of a draggable map.

The `ResponseMap` class uses [`ipywidgets`](https://pypi.org/project/ipywidgets) `Map` class dragging and zooming capabilities to capture the center of a HERE map and sends it to an observer function. This class expects a `position_handler` constructor parameter respecting the traitlets event handler [signature](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html#Registering-callbacks-to-trait-changes-in-the-kernel). In the snippet below, the map position is only displayed in an `Output` widget, each time the user interacts with the map:

In [ ]:
from here_search.demo.widgets.output import ResponseMap
from here_search.demo.api import API

api = API(api_key=api_key)
out_map = Output(height=30)

def position_handler(latitude, longitude):
    with out_map:
        out_map.replace(f'{latitude},{longitude}')
    
rmap = ResponseMap(api_key=api.api_key, 
                   center=(52.51604, 13.37691),
                   layout = {'height': '200px'},
                   position_handler=position_handler)

display(rmap, out_map)

## OneBoxSimple

`OneBoxSimple` class brings the previously described widgets together to offer both a location aware one-box and a taxonomy search experience.<br/>
For this, a asyncio task is continusouly waiting for search events, and dispatch them to the dedicated search request and response handlers:
- `OneBoxSimple` hosts an instance of `API`, a `search_center` tuple attribute providing a location context and a `language` preference attribute initialized to english.
- `TransientTextIntent` and `FormulatedTextIntent` objects materialization is sent to `API.autosuggest()` and `API.discover()` respectively.
- `PlaceTaxonomyIntent` materialization is sent to `API.browse()`.
- Autosuggest responses are sent to `handle_suggestion_list()`
- Discover and Browse responses are sent `handle_result_list()`

In the snippet below, `App` overwrites the response handlers to display them to a `ResponseMap` and an `Output` widget. The `search_center` attribute is bound to the map center:

In [ ]:
from here_search.demo.base import OneBoxSimple
from json import dumps

queue4 = Queue()
text_box = SubmittableTextBox(queue4)
terms_buttons = TermsButtons(text_box, index=-1)
taxonomy_buttons = PlaceTaxonomyButtons(queue4, PlaceTaxonomyExample.taxonomy, PlaceTaxonomyExample.icons)
rmap = ResponseMap(api_key=api.api_key, center=(52.51604, 13.37691), layout = {'height': '200px'})
out_simple = Output(height=160)

class App(OneBoxSimple):
    
    def __init__(self, queue):
        super().__init__(api=api, queue=queue)
        def handler(latitude, longitude):
            self.search_center = latitude, longitude
        rmap.set_position_handler(handler)
    
    def handle_suggestion_list(self, resp):
        with out_simple:
            out_simple.replace(dumps(resp.data, indent=2))
        terms_buttons.set(resp.terms)
        
    def handle_result_list(self, resp):
        with out_simple:
            out_simple.replace(dumps(resp.data, indent=2))
        text_box.text_w.value = ''
        terms_buttons.set([])
        rmap.display(resp)

app = App(queue4).run()
display(taxonomy_buttons, text_box, terms_buttons, rmap, out_simple)